Initial, çünkü pipeline'lı ana template daha sonra verilecek

# Linkler

- <a href="https://mvolkanyurtseven.medium.com/makine-%C3%B6%C4%9Frenimi-felsefesi-4-35964b040dac">Özet template</a>
- <a href="https://mvolkanyurtseven.medium.com/makine-%C3%B6%C4%9Frenimi-felsefesi-5-78cd290b46f2">Detaylı EDA</a>
- <a href="https://mvolkanyurtseven.medium.com/makine-%C3%B6%C4%9Frenimi-felsefesi-6-dac29cd0df59">Detay işlem sırası</a>

# Kütüphanelerin import işlemi

Burada kritik kütüphanalerden gerekli importlar yapılır. modelleme ile ilgili oalnalrı aşağıda import edeceğiz

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#mypeyxt site-packages içinde
from mypyext import dataanalysis as da
from mypyext import extensions
from mypyext import ml

# Data temini

In [ ]:
path = "..."
df=pd.read_csv(path)
df.head()

In [ ]:
#descriptive, shape ve info
df.shape
df.info()
df.describe().T

# EDA

In [ ]:
#super_info_ ile genel bilgi alalım

In [ ]:
#silinecekleri silelim

In [ ]:
#Targetın dağılımına bakalım
sns.countplot(x=df.targetkolon);

In [ ]:
# Düşük kardinalitesi olanların değerlerine bakalım
da.getColumnsInLowCardinality(df)

In [ ]:
# veri türlerini belirleyelim
target=["xxx"]
nums=["a","b"]
# cats=list(df.columns).removeItems_(nums+target,False) #extension metodum
cats=set(df.columns).difference(nums+target)
ords=["yyy"]
noms=cats.difference(ords)

nums,cats,ords,noms

In [ ]:
# korelasyon bakalım
sns.heatmap(df.select_dtypes("number").corr(), annot=True, fmt=".2f");

In [ ]:
from dython.nominal import cramers_v, correlation_ratio, associations
corrdict=associations(df,nominal_columns=cats,numerical_columns=nums,figsize=(10,10))

In [ ]:
# Targetla en yüksek korelasyonu olan kolonlara bakalım.
corr_results=corrdict["corr"] #dataframe
da.getHighestPairsOfCorrelation(corr_results,target,5)

Ödev: farklı veri tipleri arasındaki korelasyon değerlerinin değer skalası

In [ ]:
# Değişkenlerin dağılımlarına bakalım
sns.pairplot(df[nums],height=1, aspect=1.2);

In [ ]:
# outlier bakalım
df[nums].plot(kind="box", subplots = True,figsize=(6,5))
plt.tight_layout();

In [ ]:
#violin ile de bakalım, çünkü bu grafik türü yoğunluk ve olasılık dağılımını gösterir
plt.figure(figsize=(6,4))
for e,n in enumerate(nums):
    plt.subplot(1,len(nums),e+1)
    ch=sns.violinplot(y=n, data=df)
plt.tight_layout()
plt.show();

In [ ]:
#programatik oalrak da bakalım
da.outlierinfo(df,nums,imputestrategy="None",thresh=0.25)
da.outliers_IQR(df,nums,imputestrategy="None",thresh=0.25)
da.outliers_zs(df,nums,imputestrategy="None")
da.outliers_std(df,nums,imputestrategy="None")

In [ ]:
#null kontrolü
import missingno as msno
msno.bar(df, figsize=(6,4));
msno.matrix(df,figsize=(6,4));

In [ ]:
# -1, -999, NA, Tanımsız gibi null yerien geçebilecek değerler var mı diye de bakalım
da.findNullLikeValues(df)

### Target bazlı analizler

Targetın dağılımına bakalım

In [ ]:
plt.gcf().set_size_inches(4,3)
sns.countplot(x=df.targetkolon);

In [ ]:
df.targetkolon.value_counts(normalize=True)


Target bazında numeriklerin **ortalama** değerlerine bakalım.

In [ ]:
da.plotNumericsByTarget(df,"targetkolon",nums=nums,layout=(1,4),figsize=(8, 3))

Kategorik kolonlar üzerinden baktığımızda ilgili kategorilerdeki her bir değer için poztifi olma  olasılıklarına(prior probability) bakalım.

In [ ]:
plt.figure(figsize=(10,6))
da.plotTargetByCats(df, cats, "targetkolon", subplot_tpl=(2,2));

Şimdi de hayatta kalanların kategoriler bazındaki dağılımına bakalım.

In [ ]:
plt.figure(figsize=(14,8))
da.plotPositiveTargetByCats(df, cats, "targtkolon", subplot_tpl=(2,4),pos_label=1);

Her bir kategorik feature ve numerik feature çifti için target'ın ortalama değerlerine bakalım.

In [ ]:
da.plotTargetForNumCatsPairs(df,nums,cats,"target",2.4,0.9)

Son olarak da targetın diğer kategoriler bazında ortalama numerik değerlerine bakalım

In [ ]:
da.plotCategoricForNumTargetPairs(df,nums,cats,"target",2.4,0.9)

#### Target'ın iki yüzü

Hangi yönden baktığınıza göre rakamlarda çelişki, veya açıklaması zor bir durum varmış gibi görünüyor. Burada aslıdna iki ayrı durum var: **pozitiflerin yüzdesie** ve **pozitiflerin mutlak rakamı**. Bunlardan rastgele bir boyuta göre bir detay inceleme yapılabilir.

In [ ]:
df.groupby("Feature1")["targetkolon"].value_counts(normalize=True).to_frame().join(
df.groupby("Feature1")["targetkolon"].value_counts().to_frame())

Başka yapılabilecek de olur, EDA'da bakılacak şeylerin sınırı yok. şimdilik bunalr yeterli

# Varsayımların kontrolü

Model kurulacaksa o modelin gerektirdiği varsayımların kontrolü(model öncesi kontroller)

# Instance(Veri miktarı) kontrolü 

Yeterli sayıda instance(satır/kayıt) var mı kontrolü ve/veya kolon sayısı çok mu? (ML modeli kuracaksak)

- Çok az satır varsa daha fazla veri toplanması sağlanmalıdır
- Aşırı çok satır varsa kısmi sampling yapılabilir.(Detayları göreceğiz)
- Data çokluğundan emin değilseniz tamamıyla deneyin. Eğitim süresi çok uzun sürüyorsa aşamalı olarak azaltabilirsiniz.

In [ ]:
len(df)

Bu bilgi bize hangi modelleri kullanmamız konusunda ilk ipucunu da verecek:

- az veri:  high-bias/low-variance classifier (e.g. Naive Bayes, LogReg, Lineer SVM, k-NN(büyük k ile), Decision Tree(max_depth küçük,min_samples_leaf üyük), LDA/QDA). Neden?
- çok veri: low-bias/high variance classifier. Neden?

# Feature miktarı kontrolü

Yeterli sayıda kolon var mı ve/veya kolon sayısı çok mu? (ML modeli kuracaksak)
- Az kolon(feature) varsa yenileri temin edilmeye çalışılabilir
- Çok kolon varsa çeşitli boyut indirgeme ve önemli kolonları seçme yöntemleri uygulanır(Detayları sorna göreceğiz)
- Yine satırlardaki aynı mantıkla çok kolon olup olmadığında emin değilseniz önce tümüyle birlikte modelleme yapılır. Eğitim süresi uzun ise veya overfitting oluyorsa feature azaltma yöntemleri uygulanabilir.

Kolon sayısını azaltma sadece eğitim zamanını kısatlmakla kalmaz aynı zamanda overfittingi de engeller.

# Imbalancedness kontrolü

Classification problemiyse, veri setimizin labelları dengeli(balanced) mi? (Imbalanced ise train/test ayrımından sonra oversample yapılır)

# X, y ve eğitim/test setlerini oluşturma

Preprocessinge balamadan önce veri setini nasıl böleceğimize karar verelim.

train, test, validation, holdout; Bunların hangileri olacak, veriseti büyüklüğü ve zaman kısıtımıza göre karar veririz.

In [ ]:
#if imbalanced add stratify=y
from sklearn.model_selection import train_test_split


In [ ]:
df.columns

In [ ]:
X=df.iloc[:,1:]
y=df.iloc[:,0].values

In [ ]:
#if imbalanced add stratify=y
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42,stratify=y)

In [ ]:
list(map(np.shape, (X_train, X_test, y_train, y_test)))

In [ ]:
list(map(type, (X_train, X_test, y_train, y_test)))

(Opsiyonel) df, X ve y'yi silelim ki yanlışlıkla onları kullanmayalım

In [ ]:
del X
del y
del df

# Preprocessing işlemleri

Normalde bu aşamalr pipeline içine yedirilir. Biz açık açık görmek için şimdilik geçici olarak yapacağız. pipeline çinde X_train v.s yeniden oluşturup ilerleyeceğiz.

- data cleaning(Kirli data hep gelecekse, bu da pipeline içine konur, aksi halde tek seferde bitirilir)
- imputation(missing value handling) veya deletion: outlier treatten önce olmalı, IQR’ları etkiliyor
- outlier handling (Scalingden önce olmalı)
gerekiyorsa discretization(onehot öncesinde yapılmalı, ki bunlar da encode olabilsin)
feature creation/extraction
feature selection/dimension reduction
encoding(categoricler için sırası çok kritik değil, ordinallerse scalingden önce olabilir)
distribution transform for skewed data(scaling öncesinde yapmak lazım ki normal dağılıma uygun hale gelsin)
scaling
ovesampling/undersampling for imbalanced data (bu notebookta yapmayacağız)

## Data cleaning

Kirli data hep gelecekse, bu da pipeline içine konur, aksi halde tek seferde bitirilir.

- Mükerrer kayıtları silme
- Hatalı kayıtları silme
veri kalitesi problemlerini düzeltme(veri yönetişimi/data governance çalışmaları önemli):
- “Direkt Bnk”, “Direkt Bank”, “DIREKT Bank.”
- A kolonu B’den büyük olmaması gerektiği halde durum buysa
- A kolonunda bulunmaması gereken değerler varsa (Ör:il kodlarında 89, sıcaklık değeri olarak 500)
- Dönüşüm işlemleri. Ör: 100 TL → 100
- Veri tipi dönüştürme (str-->int v.s)

In [ ]:
#duplicate check for rows
len(df)-len(df.duplicated(keep=False))

In [ ]:
#duplicate check for columns
len(set(df.columns))-len(df.columns)

In [ ]:
#numeric borders, check the min-max
df.describe()

In [ ]:
#veri tipleri beklendiği gibi mi: para birimi v.s var mı diye de bakalım
df.super_info_()

## imputation

In [ ]:
# missing kontrolü
df.isnull().values.any() # NA/null değer var mı kontrol edilir. df.isnull().any().sum() # kaç kolonda null var
df.isnull().sum() #Varsa kaç adet olduğunu görmek için
df.isnull().all().sum() #tüm kolonu null olan var mı

In [ ]:
nullcols=df.columns[df.isnull().any()].tolist()
df[nullcols].describe(include="all")

In [ ]:
# imputation
df.somenumeric.fillna(df.somenumeric.median(), inplace=True)
df.somenominal.fillna(df.somenominal.mode()[0], inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

In [ ]:
# ohe
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

## Outlier handling

In [ ]:
df.somenumeric=df.somenumeric.map(lambda x:n if x>n else x)

## Discretization

In [ ]:
for o in df.select_dtypes("O").columns:
    print(df[o].value_counts())

## feature creation/extraction

## feature selection

## Feature Encoding (OHE v.s)

## Feature Transformation

## Scaling

## Oversampling/Undersampling (if imbalanced)

## Dimension reduction

# Modelleme

In [ ]:
# ilgili algoritmaya ait sınıf improt edilir

In [ ]:
# eğitim

In [ ]:
# öncelikle X_test üzerinde preprocessingleri yapalım; DİŞKAT: bilgi trainden gelmeli


In [ ]:
# tahminnleme